# Facial Keypoints - Model Training

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

### Prepare the Dataset

In [2]:
# Split the training images into a Training (80%) and Validation (20%)
# Use the images in the test directory for testing on unseen data.

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler

from facepoint.data_load import FacialKeypointsDataset
from facepoint.data_load import Rescale, RandomCrop, Normalize, ToTensor

BATCH_SIZE = 32
TRAIN_IMG_DIR = "data/training/"
TRAIN_IMG_KEYPTS = "data/training_frames_keypoints.csv"
TEST_IMG_DIR = "data/test/"
TEST_IMG_KEYPTS = "data/test_frames_keypoints.csv"


# Define a Data Transform to apply to images in the dataset.
data_transform = transforms.Compose([Rescale(250), RandomCrop(224), Normalize(), ToTensor()])


# Create training and validation data loaders.
transformed_dataset = FacialKeypointsDataset(csv_file=TRAIN_IMG_KEYPTS, 
                                             root_dir=TRAIN_IMG_DIR, 
                                             transform=data_transform)
train_sampler = SubsetRandomSampler(range(2770)) # about 80% of the training images 
validation_sampler = SubsetRandomSampler(range(2771,len(transformed_dataset)))

train_loader = DataLoader(transformed_dataset, 
                          batch_size=BATCH_SIZE, 
                          sampler=train_sampler)
validation_loader = DataLoader(transformed_dataset, 
                               batch_size=BATCH_SIZE, 
                               sampler=validation_sampler)

# Create the test dataset loader
test_dataset = FacialKeypointsDataset(csv_file=TEST_IMG_KEYPTS, 
                                      root_dir=TEST_IMG_DIR, 
                                      transform=data_transform)
test_loader = DataLoader(test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=True)

### Train the Model

In [3]:
# Get the Model

from facepoint.models import Net

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 8, kernel_size=(4, 4), stride=(1, 1))
  (conv3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(16, 24, kernel_size=(2, 2), stride=(1, 1))
  (drop1): Dropout(p=0.1, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (drop3): Dropout(p=0.3, inplace=False)
  (drop4): Dropout(p=0.4, inplace=False)
  (drop5): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=3456, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=136, bias=True)
)


In [ ]:
# Train the Model

import torch.optim as optim
import torch.nn as nn
from facepoint.model_trainer import ModelTrainer

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.005)

trainer = ModelTrainer(net, optimizer, criterion, train_loader, validation_loader)
trainer.init_weights()
trainer.train(40)

Epoch: 0, Training Loss: 0.756029702104386 | Validation Loss: 0.592774602378329
Epoch: 1, Training Loss: 0.6050292910866789 | Validation Loss: 0.5880362865031197
Epoch: 2, Training Loss: 0.5914679987335894 | Validation Loss: 0.5722955074134335
Epoch: 3, Training Loss: 0.5617326003119403 | Validation Loss: 0.6808404941462228
Epoch: 4, Training Loss: 0.6016025299522421 | Validation Loss: 0.6905672325066651
Epoch: 5, Training Loss: 0.6094057126380906 | Validation Loss: 0.6280118990393002
Epoch: 6, Training Loss: 0.6111866951203949 | Validation Loss: 0.6545986112223348
Epoch: 7, Training Loss: 0.5925168742556864 | Validation Loss: 0.7126086969710294
Epoch: 8, Training Loss: 0.6142823430390134 | Validation Loss: 0.6183506977540885
Epoch: 9, Training Loss: 0.590129230803531 | Validation Loss: 0.6063574766025874
Epoch: 10, Training Loss: 0.5867677690320067 | Validation Loss: 0.660537314829019
Epoch: 11, Training Loss: 0.6193612806418313 | Validation Loss: 0.6512539765855511
Epoch: 12, Trainin

In [ ]:
# Test on some sample data
from facepoint.utils import visualize_output, net_sample_output

test_images, test_outputs, gt_pts = net_sample_output(test_loader, net)
visualize_output(test_images, test_outputs, gt_pts, 4)

In [ ]:
# Save model
trainer.save_weights()